# Compute error

In [ ]:
from path import Path
import numpy as np
import matplotlib.pyplot as plt
import argparse
import json
from PIL import Image


def compute_errors_test(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25   ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())

    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())

    abs_diff = np.mean(np.abs(gt - pred))
    abs_rel = np.mean(np.abs(gt - pred) / gt)

    sq_rel = np.mean(((gt - pred)**2) / gt)

    return abs_rel, abs_diff, sq_rel, rmse, rmse_log, a1, a2, a3

In [ ]:
parser = argparse.ArgumentParser(description='Test octDPSNet',
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('data', metavar='DIR',
                    help='path to dataset')
parser.add_argument('--nlabel', type=int ,default=64, help='number of label')
parser.add_argument('--mindepth', type=float ,default=0.5, help='minimum depth')
parser.add_argument('--maxdepth', type=float ,default=10, help='maximum depth')
args = parser.parse_args('DATASET/demon/test'.split())
args

In [ ]:
DATA_FOLDER = args.data
ttypes = ['mvs_test.txt', 'sun3d_test.txt', 'rgbd_test.txt', 'scenes11_test.txt']
total_num = 0
for it in ttypes:
    with open(DATA_FOLDER + it + '.json') as f:
        total_num += len(json.load(f))
        


In [ ]:
basepath = Path('./')
gt_depths = sorted((basepath/'gt_depth_results').files('*.npy'))
pred_disps = sorted((basepath/'pred_results').files('*.npy'))

assert len(gt_depths) == total_num
assert len(pred_disps) == total_num

In [ ]:
# errors = np.zeros((2, 8, len(gt_depths)+4), np.float32)
errors = np.zeros((1, 8, len(gt_depths)), np.float32)
for i, (gt_fname, pred_fname) in enumerate(zip(gt_depths, pred_disps)):
    tgt_depth = np.load(gt_fname)
    output_disp = np.load(pred_fname)
    output_depth = args.mindepth * args.nlabel / output_disp
    mask = (tgt_depth <= args.maxdepth) & (tgt_depth >= args.mindepth) & (tgt_depth == tgt_depth)
    errors[0, :, i] = compute_errors_test(tgt_depth[mask], output_depth[mask])
    if i%100 == 0:
        print('Processing,,,:',i)

print('end')

In [ ]:
# errors_all[dataset_name] = errors
mean_errors = errors.mean(2)

error_names = ['abs_rel', 'abs_diff', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3']

print(str(pred_fname.dirname()))
print("Depth Results : ")
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format(*error_names))

total_num = 0
for it in ttypes:
    print('dataset:', it.split('_')[0])
    with open(DATA_FOLDER + it + '.json') as f:
        length = len(json.load(f))
        one_errors = errors[:,:,total_num:total_num+length]
        total_num += length
    print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(*one_errors.mean(2)[0]))
print('dataset:', 'ALL')
print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(*mean_errors[0]))


# Result Table

In [ ]:
import pandas as pd
from collections import OrderedDict
import numpy as np

In [ ]:
csv_fnames = OrderedDict(
    {
    r'Our ($\alpha$=0.25)':'metric_csv/a25.csv',
    r'Our ($\alpha$=0.5)':'metric_csv/a50.csv',
    r'Our ($\alpha$=0.75)':'metric_csv/a75.csv',
    r'Our ($\alpha$=0.875)':'metric_csv/a875.csv',
    r'Our ($\alpha$=0.9375)':'metric_csv/a9375.csv',
    'DPSNet':'metric_csv/dpsnet.csv',
})
# csv_fnames = OrderedDict(
#     {
#     'proposed':'metric_csv/a75.csv',
#     'no color':'metric_csv/a75_nocolor.csv',
#     'no integration':'metric_csv/a75_justAdd.csv',
#     'no alignment':'metric_csv/a75_oldsweeping.csv'
# })

# csv_fnames = OrderedDict(
#     {
#     'proposed':'metric_csv/a75.csv',
#     'no color':'metric_csv/a75_nocolor.csv',
# })
# csv_fnames = OrderedDict(
#     {
#     'a=0.25 (24)':'metric_csv/a25.csv',
#     'a=0.9375 (2)':'metric_csv/a9375.csv',
#     'dpsnet':'metric_csv/dpsnet.csv',
# })

In [ ]:
def loadDFfromCSV(fname):
    df = pd.read_csv(fname ,header=2, index_col=0)
    df.columns = df.columns.str.replace(' ', '')
    if '_' in df.index[0]:
        new_index = [it.split('_')[1] for it in df.index]
        df.index = new_index
        
    with open(fname) as f:
        header = f.readlines()[:2]
        
    return header[0].split('_gpu')[0], df 


In [ ]:
df_list = []
key_list = []
for key, fname in csv_fnames.items():
    h, df = loadDFfromCSV(fname)
    print(h)
    # add
    df_list.append(df)
    key_list.append(key)

In [ ]:
pd.options.display.precision = 4
df = pd.concat(df_list, axis=0, keys=key_list)
df = df.swaplevel().sort_index()
df = df.reindex(index=df_list[0].index,level=0)
df = df.reindex(index=key_list, level=1)
df

In [ ]:
# print(df.to_latex())

In [ ]:
def highlightDF(df, loc=None):
    attr_bold = 'font-weight: {}'.format('bold')
    attr_underline = 'text-decoration:underline'
    attrs = [attr_bold, attr_underline]
    topk = len(attrs)
    
    min_is_good = ['abs_rel', 'abs_diff', 'sq_rel', 'rms', 'log_rms']
    max_is_good = ['a1', 'a2', 'a3']

    if loc==None:
        add_attr = pd.DataFrame(np.full(df.shape, ""), index=df.index, columns=df.columns)

        for index in set(df.index.get_level_values(0)):
            _decorate(df.loc[index], add_attr.loc[index], min_is_good, topk=topk)
            _decorate(df.loc[index], add_attr.loc[index], max_is_good, topk=topk, reverse=True)

    else:
        # One index
        df=df.loc[loc]
        add_attr = pd.DataFrame(np.full(df.shape, ""), index=df.index, columns=df.columns)

        _decorate(df, add_attr, min_is_good, topk=topk)
        _decorate(df, add_attr, max_is_good, topk=topk, reverse=True)

    add_attr.replace(dict(enumerate(attrs)), inplace=True)
    func = lambda x:add_attr
    return df.style.apply(func, axis=None)

def _decorate(df, add_attr, metrics, topk=1, reverse=False):
    for metric in metrics:
        roi = df[metric]
        for i in range(topk)[::-1]:
            sorted_roi = roi.sort_values()
            if reverse:
                sorted_roi = sorted_roi[::-1]
            mask = (roi ==sorted_roi[i])
            add_attr[metric][mask]=i


In [ ]:
highlightDF(df)

In [ ]:
highlightDF(df, loc='all')

In [ ]:
# print(df.to_latex())

## ToLatex

In [ ]:
df = df.loc[['mvs', 'sun3d', 'rgbd', 'scenes11']]

#### Dataset name #######
cvt2multirow = {'mvs':r'{*}{MVS}',
               'sun3d':r'{*}{SUN3D}',
               'rgbd':r'{30px}{\centering RGB-D SLAM}',
               'scenes11':r'{*}{Scenes11}',
               }
cvt2multirow
multirow_head = r'\multirow[c]{'+str(len(index_models))+ '}'
print(multirow_head + cvt2multirow['mvs'])


topk = 2

#### Metric name #####
colums = ['Abs Rel', 'Abs Diff', 'Sq Rel', 'RMS', 'log RMS',
          '$\delta < 1.25$', '$\delta < 1.25^2$' '& $\delta < 1.25^3$']

#### Attr #####
tex_attr = {'':'', 0:r'\bfseries', 1:''}
tex_attr = {'':'', 0:r'\bfseries', 1:r'\underline'}

min_is_good = ['abs_rel', 'abs_diff', 'sq_rel', 'rms', 'log_rms']
max_is_good = ['a1', 'a2', 'a3']

add_attr = pd.DataFrame(np.full(df.shape, ""), index=df.index, columns=df.columns)
for index in set(df.index.get_level_values(0)):
    _decorate(df.loc[index], add_attr.loc[index], min_is_good, topk=topk)
    _decorate(df.loc[index], add_attr.loc[index], max_is_good, topk=topk, reverse=True)


In [ ]:

print(r'\begin{table*}')
print(r'\caption{xxxxxx}')
print(r'\centering')
print(r'\setlength{\tabcolsep}{3pt}')
# print(r'\begin{tabular}{' + '|l|' + 'r'*5 + '|rrr|' + '}')
print(r'\begin{tabular}{|cc|rrrrr|rrr|}')
print(r'\hline')
print(r'\multicolumn{2}{|c|}{} &  \multicolumn{5}{c|}{Error (smaller is better)} & \multicolumn{3}{c|}{Threshold (bigger is better)} \\')
print('Dataset & Model & ' + ' & '.join(colums) + r' \\')
print(r'\hline')
#### BODY ####
index_datasets = df.index.get_level_values(0).unique()
index_models = df.index.get_level_values(1).unique()
for idx_data in index_datasets:
    for idx_model in index_models:
        roi = df.loc[idx_data, idx_model]
        roi_attr = add_attr.loc[idx_data, idx_model]

        if idx_model == index_models[0]:
            series = [multirow_head + cvt2multirow[idx_data]]
        else:
            series = ['']

        series.append(idx_model)
        for col in df.columns:
            val = roi[col]
            attr = roi_attr[col]
            series.append('{:.4f}'.format(val))
            if tex_attr[attr] != '':
                series[-1] = tex_attr[attr] + '{' +series[-1] + '}'
        print(' & '.join(series), r'\\')
    print(r'\hline')
#### BODY END ####
print(r'\end{tabular}')
print(r'\label{tab:xxxx}')
print(r'\end{table*}')


# '& xxxxx ' * 8